<div style="display: flex; align-items: center; padding: 20px; background-color: #f0f2f6; border-radius: 10px; border: 2px solid #007bff;">
    <img src="../logo.png" style="width: 80px; height: auto; margin-right: 20px;">
    <div style="flex: 1; text-align: left;">
    <h1 style="color: #007bff; margin-bottom: 5px;">GLY 6739.017S26: Computational Seismology</h1>
    <h3 style="color: #666;">Notebook 07: Practical Demo: Trillium Compact PH + Centaur (9-Step Workflow)</h3>
    <p style="color: red;"><i>Glenn Thompson | Spring 2026</i></p>
    </div>
</div>

**Goal (in class):** Stand up a Trillium Compact PH + Nanometrics Centaur station, verify timing/sampling, generate a clear test signal (stomp), pull data in multiple ways, and visualize it with SWARM and ObsPy.

> This notebook is written to be *walk-through friendly* during a live demo.  
> Anywhere you see `TODO`, fill in your actual IPs/ports/station codes.

---

## What you need (quick checklist)

- Trillium Compact PH + power
- Centaur digitizer + power
- Ethernet cable (or switch) + laptop on same network
- Sensor cable(s) connected properly
- A SeedLink server endpoint **or** Centaur SeedLink enabled locally
- SWARM installed (or access to a computer that has it)
- Python environment with:
  - `obspy`
  - `matplotlib`
  - optional: `pyyaml`, `requests` (only if you want small helpers)

---

## Convention used in this notebook

- **Centaur IP**: `CENT_IP`
- **SeedLink host**: `SL_HOST`
- **SeedLink port**: `SL_PORT` (default often 18000)
- **Network/Station/Location**: `NET`, `STA`, `LOC`
- **Channel codes**: e.g., `HHZ`, `HHN`, `HHE` (or `EHZ`, etc.)



## 0) Fill in your station parameters (do this first)

Set these once at the top so every later cell works.

> Tip: For a quick stomp-test demo, 100 Hz is usually fine.  
> For teaching about storage vs bandwidth, also show 20 Hz and 200 Hz as contrasts.


In [ ]:
# ====== EDIT THESE ======
CENT_IP = "169.254.35.35"     # TODO: Centaur web UI / SSH IP
CENT_USER = "root"            # TODO: if different
CENT_SSH_PORT = 22

SL_HOST = CENT_IP     # TODO: SeedLink server host (could be same as Centaur)
SL_PORT = 18000               # TODO: SeedLink port

NET = "XX"                    # TODO: network code
STA = "DEMO"                  # TODO: station code
LOC = ""                      # TODO: location code (often "", "00", etc.)

# Channels you expect (edit for your sensor configuration)
CHANNELS = ["HHZ"]            # e.g., ["HHZ","HHN","HHE"] or ["EHZ"]

# Time window for streaming plots (seconds)
PLOT_SECONDS = 60

# ====== END EDIT ======

print("Centaur IP:", CENT_IP)
print("SeedLink:", f"{SL_HOST}:{SL_PORT}")
print("NSL:", NET, STA, LOC, "Channels:", CHANNELS)


## 1) Show the Centaur web interface

**In class:** open the Centaur web UI in a browser.

- URL (typical): `http://<CENT_IP>/` or `https://<CENT_IP>/`
- Show:
  - status / health page
  - GPS lock / timing
  - current sampling rates
  - ring buffer / storage
  - SeedLink / telemetry configuration (if available)
  - channel mapping (sensor → channel codes)

### Talking points
- A digitizer is a **computer + ADC + timing + IO**, not just “a box”.
- Timing is as important as waveform shape (GPS lock, NTP, PPS).
- Sample rate choices trade off **bandwidth, storage, and frequency content**.


## 2) Configure it (sample rates, channels, telemetry)

**In class:** in the web UI, change:

- sample rate (e.g., 20 → 100 Hz, or 100 → 200 Hz)
- confirm channel codes (e.g., `HHZ` vs `EHZ`)
- confirm record length / buffering and the active data paths
- confirm SeedLink output (if you are using it)

### Suggested mini-demo
1. Set **20 Hz** and explain Nyquist (10 Hz max unaliased).
2. Set **100 Hz** and explain why local noise + stomp benefits from higher rate.
3. If you go to **200 Hz**, show “bigger data” but more detail in the stomp waveform.

### Notes
- Some systems require **restart of acquisition** after changes.
- Keep a record of the “before/after” so students see cause ↔ effect.


## 3) Stomp test + validate configuration

**In class:**
- Identify where the sensor is placed and the orientation (if 3-C).
- Do a stomp test near the sensor (consistent distance).
- Immediately show:
  - amplitude response
  - whether you see clipping
  - whether sampling rate “looks” right (time axis, waveform detail)
  - whether channels are mapped correctly

### Quick checks
- Is the sensor mass centered / locked correctly?
- Does the digitizer show “data present”?
- Is GPS locked / timing stable?


## 4) Download data (from SeedLink or local MiniSEED)

There are two common pathways:

1. **SeedLink**: fetch recent data live (best for teaching near-real-time monitoring).
2. **Local files**: copy MiniSEED off the digitizer (best for offline/forensics).

We’ll do both below.


## Python setup (run once)

Make sure you have ObsPy installed in your environment.

- Conda: `conda install -c conda-forge obspy`
- Pip: `pip install obspy`

Then run:


In [ ]:
import matplotlib.pyplot as plt
from obspy import UTCDateTime, Stream
from obspy.clients.seedlink.easyseedlink import create_client
from obspy.clients.seedlink.basic_client import Client as BasicSeedLinkClient
from obspy import read

print("ObsPy imported OK.")


## 5) View SeedLink server data with SWARM

**In class (SWARM):**
1. Open SWARM
2. Add a SeedLink source:
   - Host: `SL_HOST`
   - Port: `SL_PORT`
3. Add channels for `NET.STA.LOC.CHAN` (or whatever your server uses)
4. Show:
   - waveform view
   - spectrogram
   - zooming/scaling
   - picking a time window
   - how stomp appears across channels (if 3-C)

### Talking points
- SWARM is a *monitoring* tool: quick look, fast interaction, real-time.
- For analysis + reproducibility, we often pull data into Python (next steps).


## 6) Plot SeedLink server data in ObsPy (live fetch)

### Option A: BasicSeedLinkClient `get_waveforms`

This is the simplest *pull a time window* approach.

> If your SeedLink server doesn’t support waveform extraction this way, use Option B.


In [ ]:
# Pull a recent time window from the SeedLink server
# (If your server requires a different pattern or if location code is None, adjust accordingly.)

t2 = UTCDateTime.utcnow()
t1 = t2 - PLOT_SECONDS

st = Stream()

client = BasicSeedLinkClient(SL_HOST, SL_PORT)
for cha in CHANNELS:
    try:
        # location code: use '' for blank; some servers want '--' or '??' semantics
        st += client.get_waveforms(NET, STA, LOC, cha, t1, t2)
    except Exception as e:
        print(f"Failed to get {NET}.{STA}.{LOC}.{cha}:", e)

print(st)
if len(st) > 0:
    st.merge(method=1, fill_value="interpolate")
    st.detrend("linear")
    st.filter("bandpass", freqmin=0.5, freqmax=20.0, corners=2, zerophase=True)

    st.plot(equal_scale=False)
else:
    print("No data returned. Check SeedLink settings, codes, and server availability.")


### Option B: Live streaming (short demo)

This uses an EasySeedLink client to stream for a brief period and then plot.

- Good for demonstrating “it’s alive” while you stomp.
- Less ideal for repeatable homework unless you record to disk.

> Streaming behavior varies by server configuration.


In [ ]:
import time
from obspy import Trace

received = Stream()

def on_data(trace: Trace):
    global received
    received += trace
    # Print minimal status occasionally
    print(trace.id, trace.stats.starttime, trace.stats.npts)

# Create streaming client
slc = create_client(SL_HOST, on_data, port=SL_PORT)

# Select streams
# SeedLink selectors are typically like: "NET_STA:CHAN" or just channel selectors depending on server.
# EasySeedLink uses "select_stream(net, station, selector)" where selector is channel or pattern.
for cha in CHANNELS:
    slc.select_stream(NET, STA, cha)

print("Streaming for ~15 seconds... do a stomp now.")
t_start = time.time()
slc.run()
# NOTE: create_client().run() is blocking; for a real class, you can interrupt manually (Kernel -> Interrupt)


If you interrupted the streaming cell, you can attempt to plot what was received:

In [ ]:
print(received)
if len(received) > 0:
    received.merge(method=1, fill_value="interpolate")
    received.detrend("linear")
    received.plot()
else:
    print("No streamed data captured (or cell was interrupted too early).")


## 7) SSH onto the digitizer (show it’s a Linux computer)

**In class:**
```bash
ssh root@<CENT_IP>
```

Things to show quickly:
```bash
uname -a
cat /etc/os-release
uptime
df -h
free -h
ip addr
ps aux | head
```

### Talking points
- Many digitizers run embedded Linux (or similar).
- “Field ops” often includes Linux skills: networking, storage, processes, logs.
- This is why we care about cybersecurity, passwords, and safe network practice.


## 8) Copy MiniSEED files off the digitizer (SFTP or FileZilla)

### SFTP (command line)

```bash
# From your laptop:
sftp root@<CENT_IP>

# Once connected:
pwd
ls
# navigate to the waveform directory (varies by config)
cd /path/to/miniseed

# download a directory or files:
get somefile.mseed
mget *.mseed
```

### scp (quick one-liner)

```bash
scp root@<CENT_IP>:/path/to/miniseed/*.mseed ./data/
```

### FileZilla (GUI)
- Host: `<CENT_IP>`
- Protocol: SFTP
- Username: `root`
- Password: (your credential)
- Drag-and-drop to local folder

> **Important:** The exact on-device path depends on your Centaur configuration.


## 9) Plot copied MiniSEED files with ObsPy

Once you have files locally (e.g., in a `./data/` folder), run the cells below.


In [ ]:
from pathlib import Path

DATA_DIR = Path("./data")  # TODO: change if needed
mseed_files = sorted(DATA_DIR.glob("*.mseed")) + sorted(DATA_DIR.glob("*.ms")) + sorted(DATA_DIR.glob("*.miniseed"))

print("Found", len(mseed_files), "MiniSEED files")
for f in mseed_files[:10]:
    print(" -", f)

if len(mseed_files) == 0:
    print("No files found. Copy some MiniSEED into ./data first.")


In [ ]:
if len(mseed_files) > 0:
    st_local = Stream()
    for f in mseed_files:
        try:
            st_local += read(str(f))
        except Exception as e:
            print("Failed to read", f, e)

    print(st_local)
    st_local.merge(method=1, fill_value="interpolate")
    st_local.detrend("linear")
    st_local.plot()


### Optional: quick filtering and a spectrogram

For a stomp test, a bandpass like 1–30 Hz often makes it pop visually.


In [ ]:
if 'st_local' in globals() and len(st_local) > 0:
    st_f = st_local.copy()
    st_f.filter("bandpass", freqmin=1.0, freqmax=30.0, corners=2, zerophase=True)
    st_f.plot(equal_scale=False)

    # Spectrogram for the first trace
    tr = st_f[0]
    tr.spectrogram(log=True, wlen=2.0, per_lap=0.9)
    plt.show()


# Suggested homework (pick 1–2, low friction)

You said you don’t have assignments yet — here are short options that align with the demo:

## Homework A (30–45 min): Sampling rate + Nyquist
1. Explain Nyquist in one paragraph.
2. If you record at 20 Hz, what is the highest frequency you can capture without aliasing?
3. Give two reasons you might choose 100 Hz instead of 20 Hz for volcano/seismo-acoustic monitoring.

## Homework B (45–60 min): Stomp test analysis in ObsPy
Using the provided MiniSEED file(s):
1. Plot raw waveform and a 1–30 Hz filtered version.
2. Estimate the time of the stomp (peak amplitude time).
3. Compute and report:
   - peak-to-peak amplitude
   - RMS in a 2-second window around the stomp
4. (Optional) Make a spectrogram and describe what you see.

## Homework C (60–90 min): “Digitizer is a Linux computer”
Write 10 bullet points describing:
- what commands you ran over SSH,
- what each command tells you about the station,
- one thing that could go wrong (network, storage, timing) and how you’d detect it.

---

# Instructor notes (your 2-hour morning reality)

If you have very limited setup time:
- Focus on **Steps 1–3** (web UI + config + stomp) as the core demo.
- If SeedLink is flaky, skip live and go straight to **Step 8–9** (local MiniSEED).
- SWARM is great, but don’t let it be a single point of failure.

Good luck — this is a strong, very “observatory-real” class session.
